#PROJET 1 BDLE 

##HUYNH Caroline

###Préparation

In [0]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")

# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

#Le dossier contenant les fichiers csv de IMDB
dir = "/FileStore/tables/Imdb/"

###Les Tables

Schéma d'un film

In [0]:
#----------------TABLE TITLE----------------
#la table Title affecte un identifiant id à un film
schema_title = """
          id INT, 
          title STRING, 
          imdb_index STRING, 
          kind_id INT, 
          production_year INT, 
          imdb_id INT, 
          phonetic_code STRING, 
          episode_id STRING,
          season_nr INT, 
          episode_nr INT, 
          series_years STRING, 
          md5sum STRING
        """

title = spark.read.csv(path = dir + "title.csv", schema = schema_title).persist()
title.createOrReplaceTempView("Title")

#----------------TABLE KIND_TYPE----------------
schema_kind_type = "id INT, kind STRING"

kind_type = spark.read.csv(path = dir + "kind_type.csv", schema = schema_kind_type).persist()

kind_type.createOrReplaceTempView("Kind_Type")
#kind_type.printSchema()

#----------------TABLE INFO_TYPE----------------
#La table Info_type indique la nature d'un attribut d'information. Sert dans Movie_Info, Person_Info
schema_info_type = "id INT, info STRING"

info_type = spark.read.csv(path = dir + "info_type.csv", schema = schema_info_type).persist()

info_type.createOrReplaceTempView("Info_Type")
#info_type.printSchema()

#----------------TABLE MOVIE_INFO----------------
schema_movie_info = """
    id int,
    movie_id int,
    info_type_id int,
    info string,
    note string
"""

movie_info = spark.read.csv(path = dir + "movie_info.csv", schema = schema_movie_info).persist()

movie_info.createOrReplaceTempView("Movie_Info")
#movie_info.printSchema()

#----------------TABLE MOVIE_INFO----------------
schema_cast_info = """
  id int,
  person_id int,
  movie_id int,
  person_role_id int,
  note string,
  nr_order int,
  role_id int
"""

cast_info = spark.read.csv(path = dir + "cast_info.csv", schema = schema_cast_info).persist()

cast_info.createOrReplaceTempView("Cast_Info")
#cast_info.printSchema()

Schéma d'une personne

In [0]:
#----------------TABLE CAST_INFO----------------
schema_cast_info = """
  id int,
  person_id int,
  movie_id int,
  person_role_id int,
  note string,
  nr_order int,
  role_id int
"""

cast_info = spark.read.csv(path = dir + "cast_info.csv", schema = schema_cast_info).persist()

cast_info.createOrReplaceTempView("Cast_Info")
#cast_info.printSchema()

#----------------TABLE ROLE_TYPE----------------
schema_role_type = "id INT, role STRING"

role_type = spark.read.csv(path = dir + "role_type.csv", schema = schema_role_type).persist()

role_type.createOrReplaceTempView("Role_Type")
#role_type.printSchema()

#----------------TABLE NAME----------------
#La table Name affecte un identifiant id à une personne
schema_name = """
  id int,
  name string,
  imdb_index string,
  imdb_id int,
  gender string,
  name_pcode_cf string,
  name_pcode_nf string,
  surname_pcode string,
  md5sum string
"""

name = spark.read.csv(path = dir + "name.csv", schema = schema_name).persist()

name.createOrReplaceTempView("Name")
#name.printSchema()

#----------------TABLE PERSON_INFO----------------
schema_person_info = """
  id int,
  person_id int,
  info_type_id int,
  info string,
  note string
"""

person_info = spark.read.csv(path = dir + "person_info.csv", schema = schema_person_info).persist()
person_info.createOrReplaceTempView("Person_Info")

Shéma d'une société de production

In [0]:
#----------------TABLE MOVIE_COMPANIES----------------
schema_movie_companies = """
    id int,
    movie_id int,
    company_id int,
    company_type_id int,
    note string
"""

movie_companies = spark.read.csv(path = dir + "movie_companies.csv", schema = schema_movie_companies).persist()

movie_companies.createOrReplaceTempView("Movie_Companies")
#movie_companies.printSchema()

#----------------TABLE COMPANY_NAME----------------
schema_company_name = """
    id int,
    name string,
    country_code string,
    imdb_id int,
    name_pcode_nf string,
    name_pcode_sf string,
    md5sum string
"""

company_name = spark.read.csv(path = dir + "company_name.csv", schema = schema_company_name)
company_name.createOrReplaceTempView("Company_Name")
#company_name.printSchema()

#----------------TABLE COMPANY_TYPE----------------
schema_company_type = "id INT, kind STRING"

company_type = spark.read.csv(path = dir + "company_type.csv", schema = schema_company_type).persist()
company_type.createOrReplaceTempView("Company_Type")
#company_type.printSchema()

#Tâche 1 : 
Restructurer la base de manière à définir des relations plus spécifiques. Définir au moins 4 relations concernant les personnes et 4 relations concernant les œuvres.

###Relations concernant les personnes :

(1) On va trier les death notes afin d'obtenir pour une personne, la date de sa mort, le lieu de sa mort ainsi que les causes de la mort et qui indique aussi l'age que la personne avait l'année de sa mort.

In [0]:
%sql
create or replace temp view PersonDeathGlobal as
select n.id, n.name, p3.info as birth_date, p.info as death_date, p2.info as death_notes 
from Name n, Info_type i, Person_Info p, Info_type i2, Person_Info p2, Info_type i3, Person_Info p3
where n.id = p.person_id
and p.person_id = p2.person_id
and p2.person_id = p3.person_id
and p.info_type_id = i.id
and p2.info_type_id = i2.id
and p3.info_type_id = i3.id
and i.info = 'death date' 
and i2.info = 'death notes'
and i3.info = 'birth date'
;
select * from PersonDeathGlobal limit 100;

id,name,birth_date,death_date,death_notes
1668,"Abbas, Abdullah",15 November 1906,3 November 1986,"Glendale, California, USA"
1989,"Abbott, Bud",2 October 1895,24 April 1974,"Woodland Hills, Los Angeles, California, USA (prostate cancer)"
4523,"Abrahams, Mort",26 March 1916,28 May 2009,"Sherman Oaks, California, USA (natural causes)"
6152,"Achard, Marcel",5 July 1899,4 September 1974,"Paris, France (diabetes)"
6466,"Acker, Berko",14 March 1945,1 December 1978,"East Berlin, East Germany"
6740,"Ackroyd, Jack",5 June 1890,1972,"Taunton, Somerset, UK"
7555,"Adam, Alfred",4 April 1908,7 May 1982,"Le Perreux-sur-Marne, Val-de-Marne, France (heart attack)"
8393,"Adams, Donald",20 December 1928,8 April 1996,"Norwich, England, UK"
8692,"Adams, Jonathan",14 February 1931,13 June 2005,"London, England, UK (stroke)"
9496,"Adamson, Peter",16 February 1930,17 January 2002,"Lincolnshire, England, UK (advanced stomach cancer)"


Dans cette partie, on recupere l'année de naissance et l'année de mort de la personne puis on fait une requete d'agrégation afin de définir l'age de la mort de la personne. Ensuite on remarque que dans death note, les causes de la mort sont entre parenthèse. Il suffit donc de faire un split sur le death note pour séparer le lieu de la mort de la cause.

In [0]:
%sql
create or replace temp view DeathAge as
select p.id, p.name, p.birth_date, p.death_date,
regexp_replace(death_date, "[0-9]+ [a-zA-Z]+ ","") - regexp_replace(birth_date, "[0-9]+ [a-zA-Z]+ ","")as age_of_death, 
split(death_notes, "\\(") as death_info
from PersonDeathGlobal p
;

create or replace temp view DeathCause as
select p.id, p.name, p.birth_date, p.death_date, p.age_of_death,
element_at(p.death_info,1) as death_place,
replace(element_at(p.death_info,2),")","") as death_cause
from DeathAge p
;

select *
FROM DeathCause p
limit 100;

id,name,birth_date,death_date,age_of_death,death_place,death_cause
1668,"Abbas, Abdullah",15 November 1906,3 November 1986,80.0,"Glendale, California, USA",null
1989,"Abbott, Bud",2 October 1895,24 April 1974,79.0,"Woodland Hills, Los Angeles, California, USA",prostate cancer
4523,"Abrahams, Mort",26 March 1916,28 May 2009,93.0,"Sherman Oaks, California, USA",natural causes
6152,"Achard, Marcel",5 July 1899,4 September 1974,75.0,"Paris, France",diabetes
6466,"Acker, Berko",14 March 1945,1 December 1978,33.0,"East Berlin, East Germany",null
6740,"Ackroyd, Jack",5 June 1890,1972,82.0,"Taunton, Somerset, UK",null
7555,"Adam, Alfred",4 April 1908,7 May 1982,74.0,"Le Perreux-sur-Marne, Val-de-Marne, France",heart attack
8393,"Adams, Donald",20 December 1928,8 April 1996,68.0,"Norwich, England, UK",null
8692,"Adams, Jonathan",14 February 1931,13 June 2005,74.0,"London, England, UK",stroke
9496,"Adamson, Peter",16 February 1930,17 January 2002,72.0,"Lincolnshire, England, UK",advanced stomach cancer


(2) On tri la partie salary history afin d'obtenir le salaire de l'acteur et on determine aussi si cet acteur a obtenu un profit pour ce film/serie

In [0]:
%sql
create or replace temp view PersonSalary as
select n.id, n.name, n.gender, p.info as movie_salary
from Name n, Person_Info p, Info_type i
where n.id = p.person_id
and p.info_type_id = i.id
and i.info = 'salary history'
;
select * from PersonSalary limit 100;

id,name,gender,movie_salary
1989,"Abbott, Bud",m,"_Rio Rita (1942)_ (qv)::$75,000"
1989,"Abbott, Bud",m,"_One Night in the Tropics (1940)_ (qv)::$17,500"
1989,"Abbott, Bud",m,"_""The Abbott and Costello Show"" (1952)_ (qv)::$15,000/episode"
1989,"Abbott, Bud",m,"_In the Navy (1941)_ (qv)::$25,000 + 5% of profits"
1989,"Abbott, Bud",m,"_Buck Privates (1941)_ (qv)::$25,000 + 5% of profits"
1989,"Abbott, Bud",m,"_Hold That Ghost (1941)_ (qv)::$25,000 + 5% of profits"
1989,"Abbott, Bud",m,"_Who Done It? (1942)_ (qv)::$25,000 + 5% of profits"
1989,"Abbott, Bud",m,"_Jack and the Beanstalk (1952)_ (qv)::$250,000"
1989,"Abbott, Bud",m,"_In Society (1944)_ (qv)::$40,000+%"
1989,"Abbott, Bud",m,"_Abbott and Costello Meet the Invisible Man (1951)_ (qv)::$75,000"


Le movie salaire possède le pattern suivant: film annee salaire.  
Dans un premier temps on fait un split sur :: car le salaire est précédé par "::", puis on supprime les caractères non essentiels tel que (qv) ...  
On recupere le titre du film en supprimant l'année et on récupère l'année en supprimant le titre. Le profit est obtenu en faisant un split sur le caractère +.

In [0]:
%sql
create or replace temp view PersonSalaryForFilm as
select n.id, n.name,
split(n.movie_salary,"::") as split_salary_from_movie
from PersonSalary n
;
 
create or replace temp view PersonSalaryNameMovie as
select n.id, n.name,
element_at(split_salary_from_movie,1) as movie_info,
element_at(split_salary_from_movie,2) as salary
from PersonSalaryForFilm n
;

create or replace temp view MovieDateMovieProfit as
select n.id, n.name,
regexp_replace(element_at(split(regexp_replace(n.movie_info, "\\(qv\\)", ""),"\\("),1),"_|\"","") as movie,
regexp_replace(element_at(split(regexp_replace(n.movie_info, "\\(qv\\)", ""),"\\("),2),"\\)_|[a-zA-Z]|/","") as year,
element_at(split(n.salary, "\\+|plus"),1) as salary,
element_at(split(n.salary, "\\+|plus"),2) as profit
from PersonSalaryNameMovie n
;
 
select * 
from MovieDateMovieProfit n
limit 100;

id,name,movie,year,salary,profit
1989,"Abbott, Bud",Rio Rita,1942,"$75,000",null
1989,"Abbott, Bud",One Night in the Tropics,1940,"$17,500",null
1989,"Abbott, Bud",The Abbott and Costello Show,1952,"$15,000/episode",null
1989,"Abbott, Bud",In the Navy,1941,"$25,000",5% of profits
1989,"Abbott, Bud",Buck Privates,1941,"$25,000",5% of profits
1989,"Abbott, Bud",Hold That Ghost,1941,"$25,000",5% of profits
1989,"Abbott, Bud",Who Done It?,1942,"$25,000",5% of profits
1989,"Abbott, Bud",Jack and the Beanstalk,1952,"$250,000",null
1989,"Abbott, Bud",In Society,1944,"$40,000",%
1989,"Abbott, Bud",Abbott and Costello Meet the Invisible Man,1951,"$75,000",null


(3) On tri l'information spouse, afin de séparer le nom du/de la conjoint/e, la date du mariage, le nombre d'enfant, et la raison de la fin du mariage...

In [0]:
%sql
create or replace temp view PersonSpouse as
select n.id, n.name, n.gender, p.info as spouse
from Name n, Person_Info p, Info_type i
where n.id = p.person_id
and p.info_type_id = i.id
and i.info = 'spouse'
;
select * from PersonSpouse limit 100;

id,name,gender,spouse
624,"Aaker, Lee",m,'Sharon Ann Hamilton' (22 September 1969 - March 1971) (divorced)
750,"Aames, Willie",m,'Maylo McCaslin' (qv) (15 March 1986 - present) (separated); 1 child
750,"Aames, Willie",m,'Victoria Weatherman' (15 December 1979 - 1984) (divorced); 1 child
763,"Aamund, Asger",m,'Suzanne Bjerrehuus' (qv) (? - present)
1521,"Abatantuono, Diego",m,'Rita Rabassini' (qv) (1984 - 1987) (divorced); 1 child
1989,"Abbott, Bud",m,'Betty Smith' (17 September 1918 - 24 April 1974) (his death); 2 adopted children
2038,"Abbott, Fredric",m,'Joanne Mavis Henthorn' (1956 - ?)
2133,"Abbott, Philip",m,'Jane Dufrayne' (qv) (? - 23 February 1998) (his death); 3 children
3482,"Abello, Jorge Enrique",m,'Marcela Salazar Jaramillo' (7 December 1998 - 2006); 1 child
3892,"Abineri, Daniel",m,'Claudia Rosencrantz' (qv) (? - present); 1 child


On supprime d'abord les caractères non essentiel: "(qv)" puis on fait une premiere separation sur "(" pour recuperer la nom de l'epoux/epouse. Une deuxieme separation est réalisé pour récuperer le nombre de fils que la personne possède avec ";" et enfin on separe la date de debut de mariage et de fin par "-"

In [0]:
%sql
create or replace temp view PersonSpouseSplit as
select n.id, n.name, n.gender,
regexp_replace(element_at(split(regexp_replace(n.spouse,"\\(qv\\)",""),";"),1),"\\(I\\)\'","")as wife_or_husband,
element_at(split(regexp_replace(n.spouse,"\\(qv\\)",""),";"),2)as childrens
from PersonSpouse n
;

create or replace temp view PersonMariageSplit as
select n.id, n.name, n.gender,
regexp_replace(element_at(split(n.wife_or_husband,"\\("),1),"\\'","") as wife_husband_name,
regexp_replace(element_at(split(n.wife_or_husband,"\\("),2),"\\)","") as mariage_date,
regexp_replace(element_at(split(n.wife_or_husband,"\\("),3),"\\)","") as ending_reasons,
n.childrens
from PersonSpouseSplit n
;

create or replace temp view PersonMariageStartEnd as
select n.id, n.name, n.gender, n.wife_husband_name, 
element_at(split(n.mariage_date," - "),1) as mariage_date_from, 
element_at(split(n.mariage_date," - "),2) as mariage_date_to, 
n.ending_reasons, n.childrens
from PersonMariageSplit n
;

select *
from PersonMariageStartEnd
limit 100;

id,name,gender,wife_husband_name,mariage_date_from,mariage_date_to,ending_reasons,childrens
624,"Aaker, Lee",m,Sharon Ann Hamilton,22 September 1969,March 1971,divorced,null
750,"Aames, Willie",m,Maylo McCaslin,15 March 1986,present,separated,1 child
750,"Aames, Willie",m,Victoria Weatherman,15 December 1979,1984,divorced,1 child
763,"Aamund, Asger",m,Suzanne Bjerrehuus,?,present,null,null
1521,"Abatantuono, Diego",m,Rita Rabassini,1984,1987,divorced,1 child
1989,"Abbott, Bud",m,Betty Smith,17 September 1918,24 April 1974,his death,2 adopted children
2038,"Abbott, Fredric",m,Joanne Mavis Henthorn,1956,?,null,null
2133,"Abbott, Philip",m,Jane Dufrayne,?,23 February 1998,his death,3 children
3482,"Abello, Jorge Enrique",m,Marcela Salazar Jaramillo,7 December 1998,2006,null,1 child
3892,"Abineri, Daniel",m,Claudia Rosencrantz,?,present,null,1 child


(4) Tri l'information pictorial en séparant le nom du magasine, le pays et la date de publication, le titre de l'article, le numéro du volume de l'issue et la page...

In [0]:
%sql
create or replace temp view PersonPictorial as
select n.id, n.name, n.gender, p.info as pictorial_info
from Name n, Person_Info p, Info_type i
where n.id = p.person_id
and p.info_type_id = i.id
and i.info = 'pictorial'
;
select * from PersonPictorial limit 100;

id,name,gender,pictorial_info
339,50 Cent,m,"""Metro"" (Netherlands), 4 May 2012"
339,50 Cent,m,"""Spits"" (Netherlands), 9 December 2009, by: Brunopress, ""50 cent wil duet met Susan Boyle"""
339,50 Cent,m,"""TVFilm"" (Netherlands), 5 December 2009, Vol. 25, by: Righteous Kill/Moonlight, ""Als je voor 50 cent geboren bent..."""
750,"Aames, Willie",m,"""Playboy"" (USA), November 1982, Vol. 29, Iss. 11, pg. 165, by: Arthur Knight, ""Sex In Cinema 1982"""
4805,"Abrams, J.J.",m,"""AD Rotterdams Dagblad"" (Netherlands), 22 June 2012, ""Nog meer sterren dan aan de hemel"""
11582,"Advocaat, Dick",m,"""Spits"" (Netherlands), 8 December 2009, by: Pro Shots, ""Dick Advocaat"""
11582,"Advocaat, Dick",m,"""AD Sportwereld"" (Netherlands), 16 June 2012, by: AP, ""Dick Advocaat (2 pics part of article)"""
11582,"Advocaat, Dick",m,"""AD Sportwereld"" (Netherlands), 5 June 2012, by: AFP, ""Geen tijd om rust te nemen"""
11582,"Advocaat, Dick",m,"""Metro"" (Netherlands), 22 December 2009, ""Dick Advocaat"""
11582,"Advocaat, Dick",m,"""AD Rotterdams Dagblad"" (Netherlands), 12 December 2009"


On récupère le nom du magazine grace au " suivie de ( qui separe le nom du reste des informations, on obtient ensuite le pays avec un ). Le titre lui est obtenu par les ", l'auteur est identifé grace au by: precedant l'auteur, de meme pour le Vol., Iss et Pg.

In [0]:
%sql
create or replace temp view PersonMagazineName as
select n.id, n.name,
regexp_replace(element_at(split(n.pictorial_info,"\" \\("),1),"\"","") as magazine_name,
element_at(split(n.pictorial_info,"\" \\("),2) as other_info
from PersonPictorial n
;

create or replace temp view PersonMagazineCountry as
select n.id, n.name, n.magazine_name,
element_at(split(n.other_info,"\\), "),1) as country,
element_at(split(n.other_info,"\\), "),2)as other
from PersonMagazineName n
;

create or replace temp view PersonMagazineTitle as
select n.id, n.name, n.magazine_name, n.country,
regexp_replace(element_at(split(n.other,", \""),2),"\"","") as title,
element_at(split(n.other,", \""),1) as other
from PersonMagazineCountry n
;

create or replace temp view PersonMagazineAutor as
select n.id, n.name, n.magazine_name, n.country, n.title,
element_at(split(n.other,", by: "),2) as author,
element_at(split(n.other,", by: "),1) as other
from PersonMagazineTitle n
;

create or replace temp view PersonMagazinePg as
select n.id, n.name, n.magazine_name, n.country, n.title,
element_at(split(n.other,", pg. "),2) as Pg,
element_at(split(n.other,", pg. "),1) as other
from PersonMagazineAutor n
;

create or replace temp view PersonMagazineIss as
select n.id, n.name, n.magazine_name, n.country, n.title, n.Pg,
element_at(split(n.other,", Iss. "),2) as Iss,
element_at(split(n.other,", Iss. "),1) as other
from PersonMagazinePg n
;

create or replace temp view PersonMagazineEnd as
select n.id, n.name, n.magazine_name, n.country,
element_at(split(n.other,", Vol. "),1) as date,
element_at(split(n.other,", Vol. "),2) as Vol,
n.Iss, n.Pg, n.title
from PersonMagazineIss n
;

select * 
from PersonMagazineEnd
limit 100;

id,name,magazine_name,country,date,Vol,Iss,Pg,title
339,50 Cent,Metro,Netherlands,4 May 2012,null,null,null,null
339,50 Cent,Spits,Netherlands,9 December 2009,null,null,null,50 cent wil duet met Susan Boyle
339,50 Cent,TVFilm,Netherlands,5 December 2009,25,null,null,Als je voor 50 cent geboren bent...
750,"Aames, Willie",Playboy,USA,November 1982,29,11,165,Sex In Cinema 1982
4805,"Abrams, J.J.",AD Rotterdams Dagblad,Netherlands,22 June 2012,null,null,null,Nog meer sterren dan aan de hemel
11582,"Advocaat, Dick",Spits,Netherlands,8 December 2009,null,null,null,Dick Advocaat
11582,"Advocaat, Dick",AD Sportwereld,Netherlands,16 June 2012,null,null,null,Dick Advocaat (2 pics part of article)
11582,"Advocaat, Dick",AD Sportwereld,Netherlands,5 June 2012,null,null,null,Geen tijd om rust te nemen
11582,"Advocaat, Dick",Metro,Netherlands,22 December 2009,null,null,null,Dick Advocaat
11582,"Advocaat, Dick",AD Rotterdams Dagblad,Netherlands,12 December 2009,null,null,null,null


Relations concernant les oeuvres :

(1) Tri sur l'adaptation

In [0]:
%sql
create or replace temp view TitleDetail as
select t.id, t.title, t.production_year, k.kind, it.info as property, m.info as value
from Title t, Movie_Info m, Info_type it, Kind_Type k
where t.id = m.movie_id 
and m.info_type_id = it.id
and t.kind_id = k.id
and it.info="adaption"
;
select * from TitleDetail limit 100;

id,title,production_year,kind,property,value
1714929,Basic Instinct,1992,movie,adaption,"Osborne, Richard. ""Basic Instinct (Novelization of the screenplay by Joe Eszterhas)"". (New York, New York), Signet Books, New American Library, Penguin Books, 1 March 1992, Pg. 233, (BK), ISBN-10: 0451172434"
1718377,Bed,2010,movie,adaption,"David Whitehouse. ""Bed"". Cannongate"
1776735,Circle of Iron,1978,movie,adaption,"Weverka, Robert. ""Circle of Iron (Novelization of the screenplay by Stirling Silliphant and Stanley Mann)"". (New York, New York), Warner Books, March 1979, Pg. 221, (BK), ISBN-10: 0446899283"
2033855,L'immeuble,1994,tv movie,adaption,"Tiné, Caroline. ""L'immeuble"". In: ""Novel"" (Paris, France), Albin Michel, August 1990, Pg. 200, (BK)"
2339637,Tersine dünya,1993,movie,adaption,"Kemal, Orhan. ""Tersine Dünya"". (BK)"
2339637,Tersine dünya,1993,movie,adaption,"Kemal, Orhan. ""Tersine Dünya"". (BK)"
2343638,The Adventures of Ratman,2007,movie,adaption,"Ellen Weiss & Mel Friedman. ""The Adventures of Ratman"". (BK)"
2343638,The Adventures of Ratman,2007,movie,adaption,"Ellen Weiss & Mel Friedman. ""The Adventures of Ratman"". Random House, 1990, Pg. 62, (BK)"
2407611,The Return of Swamp Thing,1989,movie,adaption,"David, Peter. ""The Return of the Swamp Thing (Novelization of the screenplay by Derek Spencer and Grand Morris)"". (New York, New York), Jove Books, May 1989, Pg. 239, (BK), ISBN-10: 0515101133"
1451885,The King's Beard,1996,episode,adaption,"Tish Sommers and Will Ryan. ""The King's Beard"". Random House, 1997, (BK), ISBN-10: 0679986332"


In [0]:
%sql
create or replace temp view MovieAdaption as
select t.id, t.title, t.production_year, t.kind,
element_at(split(t.value,", ISBN-"),2) as ISBN,
regexp_replace(element_at(split(t.value,", ISBN-"),1),"( |,|.) \\(BK\\)","") as other
from TitleDetail t;

create or replace temp view MovieAdaptionPg as
select t.id, t.title, t.production_year, t.kind, t.ISBN,
element_at(split(t.other,", Pg."),2) as Pg,
element_at(split(t.other,", Pg."),1) as split_other
from MovieAdaption t;

create or replace temp view MovieAdaptionSplitOther as
select t.id, t.title, t.production_year, t.kind,
replace(element_at(split(t.split_other,"\\. \""),1),",","") as autor,
element_at(split(t.split_other,"\\. \""),2) as other,
t.ISBN, t.Pg
from MovieAdaptionPg t;


select * from MovieAdaptionSplitOther limit 100;

id,title,production_year,kind,autor,other,ISBN,Pg
1714929,Basic Instinct,1992,movie,Osborne Richard,"Basic Instinct (Novelization of the screenplay by Joe Eszterhas)"". (New York, New York), Signet Books, New American Library, Penguin Books, 1 March 1992",10: 0451172434,233
1718377,Bed,2010,movie,David Whitehouse,"Bed"". Cannongate",null,null
1776735,Circle of Iron,1978,movie,Weverka Robert,"Circle of Iron (Novelization of the screenplay by Stirling Silliphant and Stanley Mann)"". (New York, New York), Warner Books, March 1979",10: 0446899283,221
2033855,L'immeuble,1994,tv movie,Tiné Caroline,"L'immeuble"". In: ""Novel"" (Paris, France), Albin Michel, August 1990",null,200
2339637,Tersine dünya,1993,movie,Kemal Orhan,"Tersine Dünya""",null,null
2339637,Tersine dünya,1993,movie,Kemal Orhan,"Tersine Dünya""",null,null
2343638,The Adventures of Ratman,2007,movie,Ellen Weiss & Mel Friedman,"The Adventures of Ratman""",null,null
2343638,The Adventures of Ratman,2007,movie,Ellen Weiss & Mel Friedman,"The Adventures of Ratman"". Random House, 1990",null,62
2407611,The Return of Swamp Thing,1989,movie,David Peter,"The Return of the Swamp Thing (Novelization of the screenplay by Derek Spencer and Grand Morris)"". (New York, New York), Jove Books, May 1989",10: 0515101133,239
1451885,The King's Beard,1996,episode,Tish Sommers and Will Ryan,"The King's Beard"". Random House, 1997",10: 0679986332,null


(2) Tri sur les benefices d'une production en séparant les bénéfices, le pays et la date.

In [0]:
%sql
create or replace temp view TitleDetail2 as
select t.id, t.title, t.production_year, k.kind, m.info as value
from Title t, Movie_Info m, Info_type it, Kind_Type k
where t.id = m.movie_id 
and m.info_type_id = it.id
and t.kind_id = k.id
and it.info = "gross"
;
select * from TitleDetail2 limit 100;

id,title,production_year,kind,value
1640259,3 Ninjas: High Noon at Mega Mountain,1998,movie,"$308,082 (USA) (19 April 1998)"
1640259,3 Ninjas: High Noon at Mega Mountain,1998,movie,"$150,127 (USA) (12 April 1998)"
1640259,3 Ninjas: High Noon at Mega Mountain,1998,movie,"$375,805 (USA)"
1640952,39 Pounds of Love,2005,movie,"$28,125 (USA) (22 January 2006)"
1640952,39 Pounds of Love,2005,movie,"$27,830 (USA) (15 January 2006)"
1640952,39 Pounds of Love,2005,movie,"$25,467 (USA) (18 December 2005)"
1640952,39 Pounds of Love,2005,movie,"$23,725 (USA) (11 December 2005)"
1640952,39 Pounds of Love,2005,movie,"$15,206 (USA) (4 December 2005)"
1640952,39 Pounds of Love,2005,movie,"$6,293 (USA) (27 November 2005)"
1654259,A Nightmare on Elm Street: The Dream Child,1989,movie,"$22,168,359 (USA) (29 October 1989)"


In [0]:
%sql
create or replace temp view GrossSplit as
select t.id, t.title, t.production_year, t.kind,
element_at(split(t.value," \\("),1) as gross,
regexp_replace(element_at(split(t.value," \\("),2),"\\)","") as country,
regexp_replace(element_at(split(t.value," \\("),3),"\\)","") as date_gross
from TitleDetail2 t;

select *
from GrossSplit t
limit 100;

id,title,production_year,kind,gross,country,date_gross
1640259,3 Ninjas: High Noon at Mega Mountain,1998,movie,"$308,082",USA,19 April 1998
1640259,3 Ninjas: High Noon at Mega Mountain,1998,movie,"$150,127",USA,12 April 1998
1640259,3 Ninjas: High Noon at Mega Mountain,1998,movie,"$375,805",USA,null
1640952,39 Pounds of Love,2005,movie,"$28,125",USA,22 January 2006
1640952,39 Pounds of Love,2005,movie,"$27,830",USA,15 January 2006
1640952,39 Pounds of Love,2005,movie,"$25,467",USA,18 December 2005
1640952,39 Pounds of Love,2005,movie,"$23,725",USA,11 December 2005
1640952,39 Pounds of Love,2005,movie,"$15,206",USA,4 December 2005
1640952,39 Pounds of Love,2005,movie,"$6,293",USA,27 November 2005
1654259,A Nightmare on Elm Street: The Dream Child,1989,movie,"$22,168,359",USA,29 October 1989


(3) Tri sur filming dates, en séparant le debut du tournage et la fin du tournage

In [0]:
%sql
create or replace temp view FilmingDates as
select t.id, t.title, t.production_year, k.kind, m.info as value
from Title t, Movie_Info m, Info_type it, Kind_Type k
where t.id = m.movie_id 
and m.info_type_id = it.id
and t.kind_id = k.id
and it.info = "filming dates"
;
select * from FilmingDates limit 100;

id,title,production_year,kind,value
216018,Puppetman,1987,episode,17 April 1987 -
308987,Planning Meetings January 2010,2010,episode,13 January 2010 - 20 January 2010
517511,Genie Swap,2007,episode,29 June 2007 - 26 September 2007
550835,Grillen,2012,tv series,28 May 2012 - 30 May 2012
575345,Pass the Parcel,2009,episode,24 April 2009 -
1280982,The Beatles Anthology,1995,tv series,23 June 1994 - 23 June 1994
1326752,Pretty Babies,1981,episode,17 February 1981 - (script taping date)
1602432,(#1.3),1994,episode,19 October 1994 - 19 October 1994
1640436,3.14...,2013,movie,- 2 April 2011
1640924,38 premios Sant Jordi de Cinematografía,1994,tv movie,30 May 1994 - 30 May 1994


In [0]:
%sql
create or replace temp view FilmingDatesSplit as
select t.id, t.title, t.production_year, t.kind,
element_at(split(t.value,"-"),1) as start_date,
element_at(split(t.value,"-"),2) as end_date
from FilmingDates t;

select * 
from FilmingDatesSplit t
limit 100;

id,title,production_year,kind,start_date,end_date
216018,Puppetman,1987,episode,17 April 1987,
308987,Planning Meetings January 2010,2010,episode,13 January 2010,20 January 2010
517511,Genie Swap,2007,episode,29 June 2007,26 September 2007
550835,Grillen,2012,tv series,28 May 2012,30 May 2012
575345,Pass the Parcel,2009,episode,24 April 2009,
1280982,The Beatles Anthology,1995,tv series,23 June 1994,23 June 1994
1326752,Pretty Babies,1981,episode,17 February 1981,(script taping date)
1602432,(#1.3),1994,episode,19 October 1994,19 October 1994
1640436,3.14...,2013,movie,,2 April 2011
1640924,38 premios Sant Jordi de Cinematografía,1994,tv movie,30 May 1994,30 May 1994


Tri sur les essays en récuperant dans les informations le ISSN/ISBN, l'endroit et le pays et la date où l'essay a été parus,

In [0]:
%sql
create or replace temp view Essay as
select t.id, t.title, t.production_year, k.kind, it.info as property, m.info as value
from Title t, Movie_Info m, Info_type it, Kind_Type k
where t.id = m.movie_id 
and m.info_type_id = it.id
and t.kind_id = k.id
and it.info = "essays"
;
select * from Essay limit 100;

id,title,production_year,kind,property,value
1417866,The Simpsons,1989,tv series,essays,"""Zwei gelbe Staffeln auf Silber"". ""Zack"". (Germany), Iss. 45, March 2003, Pg. 38, (MG)"
1417866,The Simpsons,1989,tv series,essays,"""Homer's 6 Best Grilling Tips"". In: ""Parade"" (USA), 23 May 2010, Pg. 5, (WNP)"
1417866,The Simpsons,1989,tv series,essays,"Dawidziak, Mark. ""'Simpsons' at 21: older and wiser? Or have we just gone immature?"". In: ""Cleveland Plain Dealer"" (USA), 18 October 2009, Pg. e1+e8, (NP)"
1417866,The Simpsons,1989,tv series,essays,"Jones, Arnold Wayne. ""Woo-Hoo! The Simpsons Celebrate 20 Years."". In: ""TV Guide"" (USA), Vol. 57, Iss. 50, 7 December 2009, Pg. 30-34, (MG)"
1417866,The Simpsons,1989,tv series,essays,"Rhodes, Joe. ""Flash! 24 'Simpsons' Stars Reveal Themselves"". In: ""TV Guide"" (USA), Vol. 48, Iss. 43, 21 October 2000, Pg. 17-33, (MG)"
1417866,The Simpsons,1989,tv series,essays,"Rhodes, Joe. ""The Divine Comedy"". In: ""TV Guide"" (USA), Vol. 51, Iss. 7, 15 February 2003, Pg. 28, (MG)"
1417866,The Simpsons,1989,tv series,essays,"Roush, Matt. ""Mmmmm, Simpsons"". In: ""TV Guide"" (USA), Vol. 51, Iss. 7, 15 February 2003, Pg. 31, (MG)"
1417866,The Simpsons,1989,tv series,essays,"Solomon, Michael. ""They're Yellow But They're Not Chicken"". In: ""TV Guide"" (USA), Vol. 51, Iss. 7, 15 February 2003, Pg. 26-27, (MG)"
1417866,The Simpsons,1989,tv series,essays,"Tucker, Ken. ""The Family Dynamic"". In: ""Entertainment Weekly"" (USA), Vol. 1, Iss. 694, 7 February 2003, Pg. 28-36, (MG)"
1714929,Basic Instinct,1992,movie,essays,"Sauter, Michael. ""Sexual Dealings"". In: ""Entertainment Weekly"" (USA), Vol. 1, Iss. 645, 22 March 2002, Pg. 116, (MG)"


In [0]:
%sql
create or replace temp view EssaySplit as
select t.id, t.title, t.production_year, t.kind,
element_at(split(t.value,"ISBN|ISSN"),1) as other,
regexp_replace(element_at(split(t.value,"ISBN|ISSN"),2),"-","") as ISBN
from Essay t
;

create or replace temp view EssaySplit2 as
select t.id, t.title, t.production_year, t.kind,
element_at(split(t.other,"In: \""),1) as other1,
element_at(split(element_at(split(t.other,"In: \""),2),"\" \\(|\", "),1) as essay_in,
element_at(split(element_at(split(t.other,"In: \""),2),"\" \\(|\", "),2) as essay_info,
t.ISBN
from EssaySplit t
;


-- Suppression d'informations pas intéressante...
create or replace temp view EssaySplit3 as
select t.id, t.title, t.production_year, t.kind, t.essay_in, 
regexp_replace(t.essay_info,", Vol\. [0-9]+,| Pg. [e]*[0-9]+[ ]*[+-]*[ ]*[e]*[0-9]*|[,,]+ \\([MGBKWNP0-9]+\\)+|Iss. [0-9]+[/]*[0-9]*","") as clean_essay_info,
t.ISBN
from EssaySplit2 t
;

create or replace temp view EssayDate as
select t.id, t.title, t.production_year, t.kind,
regexp_replace(regexp_replace(t.clean_essay_info,"^((?!([0-9]* (January|February|March|April|May|June|July|August|September|October|November|December)* [0-9]*)).)*",""),",","") as date_publish,
element_at(split(regexp_replace(t.clean_essay_info, "\\), |\\) ,|\\(|\\)|, [0-9]|Au","#####"),"#####"),1) as country_publish, t.essay_in,
t.ISBN
from EssaySplit3 t
;

select * 
from EssayDate
limit 100;

id,title,production_year,kind,date_publish,country_publish,essay_in,ISBN
1417866,The Simpsons,1989,tv series,null,null,null,null
1417866,The Simpsons,1989,tv series,23 May 2010,USA,Parade,null
1417866,The Simpsons,1989,tv series,18 October 2009,USA,Cleveland Plain Dealer,null
1417866,The Simpsons,1989,tv series,7 December 2009,USA,TV Guide,null
1417866,The Simpsons,1989,tv series,21 October 2000,USA,TV Guide,null
1417866,The Simpsons,1989,tv series,15 February 2003,USA,TV Guide,null
1417866,The Simpsons,1989,tv series,15 February 2003,USA,TV Guide,null
1417866,The Simpsons,1989,tv series,15 February 2003,USA,TV Guide,null
1417866,The Simpsons,1989,tv series,7 February 2003,USA,Entertainment Weekly,null
1714929,Basic Instinct,1992,movie,22 March 2002,USA,Entertainment Weekly,null


##Tâche 2: 
Proposer un schéma en étoile où un fait est une association entre une personne, une œuvre et un rôle. Proposer des dimensions sur les personnes et les œuvres. Proposer au moins une dimension hiérarchique ayant 3 niveaux.

__Schéma en étoile :__

Le fait: (association entre une personne, une oeuvre, un rôle)  
_CastFromMovie_(cast_id, p_id, p_name, release_date, movie_id, movie_title)  
_Date_(date, jour, mois, annee)  
_MovieType_(movieTitle, movieGenre)  
_PersonRole_(personName, personRole)

In [0]:
%sql
create or replace temp view CastFromMovie as
select c.id as cast_id, c.person_id as p_id, n.name, m.info as release_date, c.movie_id, t.title
from Cast_Info c, Name n, Title t, Info_type i, Person_info p, Movie_Info m
where n.id = c.person_id
and c.movie_id = t.id
and c.person_id = p.person_id
and m.info_type_id = i.id
and i.info = "LD release date"
;
 
select * from CastFromMovie;

cast_id,p_id,name,release_date,movie_id,title
16255404,2202581,"Krijgsman, Bianca",1988,434,Brand in Mokum
16255404,2202581,"Krijgsman, Bianca",1993,434,Brand in Mokum
16255404,2202581,"Krijgsman, Bianca",1986,434,Brand in Mokum
16255404,2202581,"Krijgsman, Bianca",January 1995,434,Brand in Mokum
16255404,2202581,"Krijgsman, Bianca",6 December 1995,434,Brand in Mokum
16255404,2202581,"Krijgsman, Bianca",9 September 1992,434,Brand in Mokum
16255404,2202581,"Krijgsman, Bianca",29 January 1992,434,Brand in Mokum
16255404,2202581,"Krijgsman, Bianca",10 March 1993,434,Brand in Mokum
16255404,2202581,"Krijgsman, Bianca",1984,434,Brand in Mokum
16255404,2202581,"Krijgsman, Bianca",1984,434,Brand in Mokum


In [0]:
%sql
create or replace temporary view Date as
select distinct d.release_date,
regexp_replace(d.release_date,"[a-zA-Z]*( )*([0-9]{4})*","") as day,
regexp_replace(d.release_date,"[0-9]| ","") as month,
regexp_replace(d.release_date,"[0-9]* [a-zA-Z]*|[a-zA-Z]*","") as year
from CastFromMovie d;

create or replace temporary view DateSplit as
select distinct d.release_date,
nullif(d.day,'') as day,
nullif(d.month,'') as month,
nullif(d.year,'') as year
from Date d;


select *
from DateSplit;

release_date,day,month,year
9 September 1992,9,September,1992
3 June 1992,3,June,1992
1991,null,null,1991
1992,null,null,1992
19 August 1997,19,August,1997
March 1993,null,March,1993
January 1995,null,January,1995
9 June 1993,9,June,1993
8 June 1998,8,June,1998
2 November 1994,2,November,1994


In [0]:
%sql
create or replace temporary view MovieType as
select distinct d.title, m.info as movie_type
from CastFromMovie d, Info_Type i, Movie_Info m
where d.movie_id = m.movie_id
and m.info_type_id = i.id
and i.info = "genres";

select * from MovieType;

title,movie_type
Balash Be-Yerushalayim,Drama
Banegi Apni Baat,Drama
Beautiful You HDTV,Talk-Show
Canarias Noticias,News
Cop Shows Anonymous,Comedy
Danmarks klogeste barn,Family
De bende van Wim,Reality-TV
Ein Sofa in Berlin - Tulpe und Conrady laden ein,Talk-Show
El amor tiene cara de mujer,Drama
El pantano,Mystery


In [0]:
%sql
create or replace temporary view RolePerson as
select distinct d.name, i.role as person_role
from CastFromMovie d, Role_Type i, Cast_Info m
where d.movie_id = m.movie_id
and m.role_id = i.id;

select * from RolePerson;

name,person_role
"Krijgsman, Bianca",editor
"Luca, Loes",editor
"Bokma, Pierre",cinematographer
"Rijxman, Lineke",editor
"Duin, André van",cinematographer
"Tefsen, Carry",actress
"Kas, Ton",actress
"Arean, Jenny",editor
"Schwartzman, Jennica",actor
"Schwartzman, Jennica",editor


dimension sur les oeuvres:

In [0]:
%sql
select distinct year, month, day, count(*) as nb_release from CastFromMovie v, DateSplit d
where v.release_date = d.release_date
group by rollup (year, month, day)
order by year, month, day

year,month,day,nb_release
null,null,null,150059800
1984,null,null,5456720
1986,null,null,5456720
1988,null,null,5456720
1989,null,null,2728360
1990,null,null,5456720
1990,null,null,2728360
1990,May,null,2728360
1991,null,null,8185080
1991,null,null,5456720


## Tâche 3:
Ajouter des propriétés pour une personne, une œuvre ou un lieu.

Sur une oeuvre =>

In [0]:
dirNew = "/FileStore/tables/tsv/"
schema_title = """
          u_id int, 
          b_id int, 
          rating int
        """

rating = spark.read.csv(path = dirNew + "ratings.csv", schema = schema_title).persist()
rating.createOrReplaceTempView("Ratings")
display(rating)

u_id,b_id,rating
null,null,null
276747,4780,4
276747,1837,4
276747,6277,3
276762,7819,1
276762,4885,3
276772,27222,2
276772,33829,5
276772,83629,5
276786,246867,3


In [0]:
%sql
create or replace temporary view RatingB as
select distinct r.u_id, r.b_id, r.rating, m.info_type_id, m.info
FROM Ratings r, Movie_Info m
where r.b_id = m.movie_id;
select * from RatingB;

u_id,b_id,rating,info_type_id,info
217958,1680,4,16,USA:11 May 2011
152946,1841,3,16,Spain:20 March 2005
34600,1841,3,16,Spain:20 March 2005
213839,2907,2,16,Spain:28 September 2002
47786,3342,3,3,Music
2719,3342,3,4,English
56399,3345,5,16,USA:23 August 2001
253711,3544,2,16,USA:17 September 2004
157655,3544,3,16,USA:17 September 2004
118434,3544,2,16,USA:17 September 2004


Sur une personne =>

In [0]:
dirNew = "/FileStore/tables/tsv/"
schema_title = """
          u_id int, 
          country string, 
          age int
        """

users = spark.read.csv(path = dirNew + "users.csv", schema = schema_title).persist()
users.createOrReplaceTempView("Users")
display(users)

u_id,country,age
null,country,null
100004,usa,0
100009,canada,49
10001,usa,47
100029,germany,0
10003,usa,20
100035,canada,0
100043,usa,36
100046,usa,14
100053,usa,31


In [0]:
%sql
create or replace temporary view PersonUser as
select distinct r.u_id, n.name, r.country, r.age, p.info
FROM Users r, Person_info p, Info_type i, Name n
where r.u_id = p.person_id
and i.id = p.info_type_id
and i.info = "birth date"
and n.id = p.person_id;
select * from PersonUser;

u_id,name,country,age,info
17707,"Akerblom, Erik",usa,0,22 February 1961
42573,"Anderson, Robert Mailer",unknown,78,1968
53506,"Arana, Facundo",usa,0,31 March 1972
54564,"Arcand, Gabriel",usa,34,4 June 1949
123244,"Benedict, Richard",usa,41,8 January 1920
123754,"Benham, Harry",usa,0,26 February 1884
125917,"Benoni, Stepán",usa,33,23 August 1984
128557,"Berg, Wolf-Dietrich",united kingdom,0,17 May 1944
128669,"Bergdahl, Victor",canada,22,12 December 1878
135681,"Besson, Pierre",united kingdom,0,4 April 1967


Pour un lieu =>

J'ai rencontré quelques problème concernant ce fichier .txt, en effet, j'ai eu du mal à délimiter chaques elements du fichier car c'est un fichier contenant enormément d'information, ce qui nous fait un rendu pas très satisfaisant par rapport au rendu souhaité.

In [0]:
dirNew = "/FileStore/tables/tsv/"
schema_title = """
          iso string, 
          iso3 string, 
          iso_num string,
          flips string,
          country string,
          capital string,
          area_in_km double,
          population double,
          continent string,
          tld string,
          currency_code double,
          currency_name string,
          phone string,
          postal1 string,
          code1 string,
          format string,
          postal2 string,
          code2 string,
          regex string,
          languages string,
          geonamid string,
          neighbor string,
          equivalent_fips_code string
        """


users = spark.read.csv(path = dirNew + "countryInfo2.txt", schema = schema_title).persist()
users.createOrReplaceTempView("countryInfo")
display(users)

iso,iso3,iso_num,flips,country,capital,area_in_km,population,continent,tld,currency_code,currency_name,phone,postal1,code1,format,postal2,code2,regex,languages,geonamid,neighbor,equivalent_fips_code
#ISO ISO3 ISO-Numeric fips Country Capital Area(in sq km) Population Continent tld CurrencyCode CurrencyName Phone Postal Code Format Postal Code Regex Languages geonameid neighbours EquivalentFipsCode,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AD AND 020 AN Andorra Andorra la Vella 468 84000 EU .ad EUR Euro 376 AD### ^(?:AD)*(\d{3})$ ca 3041565 ES,FR,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AE ARE 784 AE United Arab Emirates Abu Dhabi 82880 4975593 AS .ae AED Dirham 971 ar-AE,fa,en,hi,ur 290557 SA,OM,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AF AFG 004 AF Afghanistan Kabul 647500 29121286 AS .af AFN Afghani 93 fa-AF,ps,uz-AF,tk 1149361 TM,CN,IR,null,null,UZ,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AG ATG 028 AC Antigua and Barbuda St. John's 443 86754 NA .ag XCD Dollar +1-268 en-AG 3576396,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AI AIA 660 AV Anguilla The Valley 102 13254 NA .ai XCD Dollar +1-264 en-AI 3573511,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AL ALB 008 AL Albania Tirana 28748 2986952 EU .al ALL Lek 355 #### ^(\d{4})$ sq,el 783754 MK,GR,ME,RS,XK,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AM ARM 051 AM Armenia Yerevan 29800 2968000 AS .am AMD Dram 374 ###### ^(\d{6})$ hy 174982 GE,IR,AZ,TR,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AO AGO 024 AO Angola Luanda 1246700 13068161 AF .ao AOA Kwanza 244 pt-AO 3351879 CD,NA,ZM,CG,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AQ ATA 010 AY Antarctica 14000000 0 AN .aq 6697173,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


##Tâche 4:
Essayer de traiter la totalité des données de IMDB (pas seulement l’extrait sample01), et décrire très brièvement les difficultés rencontrées.

Il est difficile de traiter la totalité des données IMDB car il y a une infinité de données et les traiter d'un seul coup qui parait presque impossible car les données ne sont pas filtré dans le cas des tables info_type, role_type ...